# k-Means Clustering mit italienischen Weinen

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_20/wine_clustering_kmeans.ipynb)

In diesem Notebook lernen wir k-Means Clustering praktisch kennen. Wir verwenden italienische Weine mit verschiedenen chemischen Eigenschaften und schauen, ob k-Means die 3 Produzenten automatisch findet.

Der Wine-Datensatz hat 13 chemische Eigenschaften als Features. Wir verwenden zunächst nur zwei davon und machen mit diesen beiden das Clustering. Am Ende des Notebooks berechnen wir dann ein weiteres Clustering auf allen 13 Features zum Vergleich.

**Ziel:** Verstehen, wie k-Means funktioniert und warum Skalierung bei verschiedenen Einheiten kritisch ist.

## 1. Bibliotheken importieren und Daten laden

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, silhouette_score

# Wein-Daten laden
wine = load_wine()
print("Wine-Dataset geladen!")
print(f"Anzahl Weine: {wine.data.shape[0]}")
print(f"Anzahl chemische Eigenschaften: {wine.data.shape[1]}")
print(f"Produzenten: {wine.target_names}")
print(f"\nErste 5 Eigenschaften: {wine.feature_names[:5]}")

## 2. Das Skalierungs-Problem verstehen

Wir schauen uns zwei Eigenschaften mit **völlig verschiedenen Skalen** an:

In [ ]:
# Zwei Features mit sehr verschiedenen Skalen wählen
X = wine.data[:, [0, 12]]  # Alkohol und Proline
feature_names = ['Alkohol (%)', 'Proline (mg/L)']

print("🍷 Wertebereich der chemischen Eigenschaften:")
print(f"Alkohol: {X[:, 0].min():.1f} - {X[:, 0].max():.1f} %")
print(f"Proline: {X[:, 1].min():.0f} - {X[:, 1].max():.0f} mg/L")
print("\n⚠️ RIESIGE Unterschiede in den Wertebereichen!")
print("Proline ist 100x größer als Alkohol!")
print("Ohne Skalierung würde k-Means nur nach Proline clustern.")

## 3. Originaldaten visualisieren

Zuerst schauen wir uns die echten Produzenten an:

In [ ]:
# Originaldaten mit echten Produzenten plotten
plt.figure(figsize=(12, 5))

# Plot 1: Normale Skala
plt.subplot(1, 2, 1)
colors = ['red', 'blue', 'green']
producer_names = ['Produzent A', 'Produzent B', 'Produzent C']
for i, producer in enumerate(producer_names):
    mask = wine.target == i
    plt.scatter(X[mask, 0], X[mask, 1], c=colors[i], label=producer, alpha=0.7, s=50)

plt.xlabel('Alkohol (%)')
plt.ylabel('Proline (mg/L)')
plt.title('Wein-Daten: Echte Produzenten')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Log-Skala um das Problem zu zeigen
plt.subplot(1, 2, 2)
for i, producer in enumerate(producer_names):
    mask = wine.target == i
    plt.scatter(X[mask, 0], X[mask, 1], c=colors[i], label=producer, alpha=0.7, s=50)

plt.xlabel('Alkohol (%)')
plt.ylabel('Proline (mg/L)')
plt.yscale('log')  # Log-Skala für Proline
plt.title('Mit Log-Skala (bessere Sicht)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Wir sehen drei relativ gut getrennte Gruppen.")
print("Kann k-Means diese automatisch finden?")
print("Problem: Die y-Achse ist viel größer als die x-Achse!")

## 4. Daten skalieren

**Kritisch wichtig:** k-Means ist sehr empfindlich gegenüber unterschiedlichen Wertebereichen!

In [ ]:
# Daten skalieren (Mittelwert 0, Standardabweichung 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Nach der Skalierung:")
print(f"Alkohol: Mittelwert={X_scaled[:, 0].mean():.3f}, Std={X_scaled[:, 0].std():.3f}")
print(f"Proline: Mittelwert={X_scaled[:, 1].mean():.3f}, Std={X_scaled[:, 1].std():.3f}")
print("\n🎯 Jetzt haben beide Features den gleichen Wertebereich!")
print("Beide haben Mittelwert 0 und Standardabweichung 1.")

# Skalierte Daten visualisieren
plt.figure(figsize=(10, 6))
for i, producer in enumerate(producer_names):
    mask = wine.target == i
    plt.scatter(X_scaled[mask, 0], X_scaled[mask, 1], c=colors[i], label=producer, alpha=0.7, s=50)

plt.xlabel('Alkohol (skaliert)')
plt.ylabel('Proline (skaliert)')
plt.title('Wein-Daten: Skaliert')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Jetzt sind beide Achsen gleich wichtig für k-Means!")

## 5. k-Means mit k=3 anwenden

Da wir wissen, dass es 3 Produzenten gibt, probieren wir k=3:

In [ ]:
# k-Means mit k=3 anwenden
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
centers = kmeans.cluster_centers_

print("🤖 k-Means Ergebnisse:")
print("Cluster-Zentren (skaliert):")
for i, center in enumerate(centers):
    print(f"  Cluster {i}: [{center[0]:.2f}, {center[1]:.2f}]")

# Wie viele Weine in jedem Cluster?
unique, counts = np.unique(cluster_labels, return_counts=True)
print("\n📊 Anzahl Weine pro Cluster:")
for cluster, count in zip(unique, counts):
    print(f"  Cluster {cluster}: {count} Weine")

print("\n🎯 Zur Erinnerung - echte Verteilung:")
unique_real, counts_real = np.unique(wine.target, return_counts=True)
for prod, count in zip(producer_names, counts_real):
    print(f"  {prod}: {count} Weine")

## 6. Ergebnisse visualisieren

Vergleichen wir die echten Produzenten mit den k-Means Clustern:

In [ ]:
# Drei Plots nebeneinander
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Echte Produzenten (Original-Skala)
for i, producer in enumerate(producer_names):
    mask = wine.target == i
    axes[0].scatter(X[mask, 0], X[mask, 1], c=colors[i], label=producer, alpha=0.7, s=50)
axes[0].set_xlabel('Alkohol (%)')
axes[0].set_ylabel('Proline (mg/L)')
axes[0].set_title('Echte Produzenten')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: k-Means Cluster (Original-Skala für besseres Verständnis)
scatter = axes[1].scatter(X[:, 0], X[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7, s=50)
# Zentren zurück-transformieren für bessere Anschauung
centers_orig = scaler.inverse_transform(centers)
axes[1].scatter(centers_orig[:, 0], centers_orig[:, 1], c='red', 
               marker='x', s=200, linewidths=3, label='Cluster-Zentren')
axes[1].set_xlabel('Alkohol (%)')
axes[1].set_ylabel('Proline (mg/L)')
axes[1].set_title('k-Means Cluster (k=3)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1], label='Cluster')

# Plot 3: Skalierte Daten mit Clustern
scatter2 = axes[2].scatter(X_scaled[:, 0], X_scaled[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7, s=50)
axes[2].scatter(centers[:, 0], centers[:, 1], c='red', 
               marker='x', s=200, linewidths=3, label='Cluster-Zentren')
axes[2].set_xlabel('Alkohol (skaliert)')
axes[2].set_ylabel('Proline (skaliert)')
axes[2].set_title('k-Means auf skalierten Daten')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

print("👀 Beobachtungen:")
print("• Links: Wie die Daten wirklich aussehen")
print("• Mitte: Was k-Means gefunden hat (auf normale Skala zurück projiziert)")
print("• Rechts: Wie k-Means die Daten 'sieht' (skaliert)")

## 7. Ergebnisse bewerten

Wie gut hat k-Means die echten Produzenten gefunden?

In [ ]:
# Bewertungsmetriken berechnen
ari = adjusted_rand_score(wine.target, cluster_labels)
sil_score = silhouette_score(X_scaled, cluster_labels)

print("📊 Bewertung der Cluster-Qualität:")
print(f"Adjusted Rand Index (ARI): {ari:.3f}")
print(f"Silhouette Score: {sil_score:.3f}")
print()
print("🔍 Interpretation:")
print(f"• ARI = {ari:.3f}: ", end="")
if ari > 0.8:
    print("Sehr gut! k-Means findet die echten Produzenten sehr ähnlich.")
elif ari > 0.6:
    print("Gut! k-Means findet die echten Produzenten ziemlich ähnlich.")
elif ari > 0.3:
    print("Okay. k-Means findet teilweise die echten Produzenten.")
else:
    print("Nicht so gut. k-Means findet die echten Produzenten nicht gut.")

print(f"• Silhouette = {sil_score:.3f}: ", end="")
if sil_score > 0.7:
    print("Sehr gut getrennte Cluster!")
elif sil_score > 0.5:
    print("Gut getrennte Cluster.")
elif sil_score > 0.3:
    print("Mäßig getrennte Cluster.")
else:
    print("Schlecht getrennte Cluster.")

## 8. Experiment: Warum Skalierung so wichtig ist

Schauen wir, was passiert, wenn wir die Daten **NICHT** skalieren:

In [ ]:
# k-Means OHNE Skalierung
print("🧪 Experiment: k-Means ohne Skalierung")
print("Erinnerung: Alkohol (11-15%) vs. Proline (278-1680 mg/L)")

kmeans_unscaled = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels_unscaled = kmeans_unscaled.fit_predict(X)

# Vergleich visualisieren
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Mit Skalierung
scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7, s=50)
centers_orig = scaler.inverse_transform(centers)
axes[0].scatter(centers_orig[:, 0], centers_orig[:, 1], c='red', 
               marker='x', s=200, linewidths=3, label='Zentren')
axes[0].set_xlabel('Alkohol (%)')
axes[0].set_ylabel('Proline (mg/L)')
axes[0].set_title('MIT Skalierung')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0])

# Ohne Skalierung
scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=cluster_labels_unscaled, cmap='viridis', alpha=0.7, s=50)
axes[1].scatter(kmeans_unscaled.cluster_centers_[:, 0], kmeans_unscaled.cluster_centers_[:, 1], 
               c='red', marker='x', s=200, linewidths=3, label='Zentren')
axes[1].set_xlabel('Alkohol (%)')
axes[1].set_ylabel('Proline (mg/L)')
axes[1].set_title('OHNE Skalierung')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

# Metriken vergleichen
ari_unscaled = adjusted_rand_score(wine.target, cluster_labels_unscaled)
sil_unscaled = silhouette_score(X, cluster_labels_unscaled)

print("\n📊 Vergleich der Metriken:")
print(f"{'Methode':<20} {'ARI':<8} {'Silhouette':<12}")
print("-" * 40)
print(f"{'✅ Mit Skalierung':<20} {ari:.3f}{'':>4} {sil_score:.3f}")
print(f"{'❌ Ohne Skalierung':<20} {ari_unscaled:.3f}{'':>4} {sil_unscaled:.3f}")

print("\n🤔 Interessante Beobachtung:")
if sil_score < sil_unscaled:
    print("Der Silhouette Score ist MIT Skalierung schlechter!")
    print("\n💡 Warum passiert das?")
    print("• OHNE Skalierung: Proline dominiert → k-Means clustert hauptsächlich nach Proline")
    print("• Da Proline allein schon gute Trennung zeigt, wirken die Cluster 'kompakter'")
    print("• MIT Skalierung: Beide Features sind gleichberechtigt → komplexere Entscheidungen")
    print("• Das kann zu Clustern führen, die weniger 'rund' sind")
    print("\n🎯 Was ist besser?")
    print("• ARI (Adjusted Rand Index) ist wichtiger - er vergleicht mit den echten Gruppen!")
    print(f"• ARI mit Skalierung ({ari:.3f}) vs. ohne ({ari_unscaled:.3f})")
    print("• Skalierung ist trotzdem richtig - sie verhindert, dass ein Feature dominiert")
    print("• Bei mehr als 2 Features wäre der Effekt noch deutlicher!")
else:
    print("Skalierung hilft auch beim Silhouette Score!")
    
print("\n💡 Lernerfahrung:")
print("• Verschiedene Metriken können verschiedene Geschichten erzählen")
print("• Skalierung ist ein Preprocessing-Schritt, der fair sein soll")
print("• Ohne Skalierung ist das Ergebnis von den Einheiten abhängig (unfair!)")
print("• Mit Skalierung behandeln wir alle Features gleich (fair!)")

## 9. Bonus: k-Means mit allen 13 Features

Bisher haben wir nur 2 Features verwendet (Alkohol und Proline). Aber der Wine-Datensatz hat **13 chemische Eigenschaften**! Was passiert, wenn wir alle nutzen?

**Hypothese:** Mit mehr Information sollte k-Means die Produzenten besser finden können.

In [ ]:
# k-Means mit ALLEN 13 Features
print("🚀 k-Means mit allen 13 chemischen Eigenschaften:")
print(f"Features: {wine.feature_names}")
print()

# Alle Daten skalieren
X_all = wine.data  # Alle 13 Features
scaler_all = StandardScaler()
X_all_scaled = scaler_all.fit_transform(X_all)

print("📊 Daten-Info:")
print(f"   • Anzahl Features: {X_all.shape[1]}")
print(f"   • Anzahl Weine: {X_all.shape[0]}")
print(f"   • Nach Skalierung: Mittelwerte ≈ {X_all_scaled.mean():.3f}, Std ≈ {X_all_scaled.std():.3f}")

# k-Means mit allen Features anwenden
kmeans_all = KMeans(n_clusters=3, random_state=42, n_init=10)
cluster_labels_all = kmeans_all.fit_predict(X_all_scaled)

# Metriken berechnen
ari_all = adjusted_rand_score(wine.target, cluster_labels_all)
sil_all = silhouette_score(X_all_scaled, cluster_labels_all)

print("\n🎯 Ergebnisse mit allen 13 Features:")
print(f"   • ARI: {ari_all:.3f}")
print(f"   • Silhouette Score: {sil_all:.3f}")

In [ ]:
# Vergleich aller drei Ansätze visualisieren
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Echte Produzenten (als Referenz)
for i, producer in enumerate(producer_names):
    mask = wine.target == i
    axes[0].scatter(X[mask, 0], X[mask, 1], c=colors[i], label=producer, alpha=0.7, s=50)
axes[0].set_xlabel('Alkohol (%)')
axes[0].set_ylabel('Proline (mg/L)')
axes[0].set_title('Echte Produzenten\n(Ground Truth)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. k-Means mit 2 Features (bisheriges Ergebnis)
scatter1 = axes[1].scatter(X[:, 0], X[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7, s=50)
axes[1].set_xlabel('Alkohol (%)')
axes[1].set_ylabel('Proline (mg/L)')
axes[1].set_title(f'k-Means (2 Features)\nARI: {ari:.3f}, Sil: {sil_score:.3f}')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[1], label='Cluster')

# 3. k-Means mit allen 13 Features (projiziert auf Alkohol/Proline für Visualisierung)
scatter2 = axes[2].scatter(X[:, 0], X[:, 1], c=cluster_labels_all, cmap='viridis', alpha=0.7, s=50)
axes[2].set_xlabel('Alkohol (%)')
axes[2].set_ylabel('Proline (mg/L)')
axes[2].set_title(f'k-Means (alle 13 Features)\nARI: {ari_all:.3f}, Sil: {sil_all:.3f}')
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

print("👀 Beobachtung:")
print("   Obwohl wir nur Alkohol vs. Proline sehen, nutzt der rechte Plot")
print("   alle 13 Features für das Clustering. Die Farben zeigen das Ergebnis!")

In [ ]:
# Detaillierter Vergleich der drei Ansätze
print("📊 Detaillierter Vergleich aller Ansätze:")
print()
print(f"{'Methode':<25} {'Features':<10} {'ARI':<8} {'Silhouette':<12} {'Bewertung'}")
print("-" * 70)

# ARI-Bewertung
def get_ari_rating(ari_val):
    if ari_val > 0.8: 
        return "Exzellent"
    elif ari_val > 0.6: 
        return "Sehr gut"
    elif ari_val > 0.4: 
        return "Gut"
    elif ari_val > 0.2: 
        return "Mäßig"
    else: 
        return "Schlecht"

print(f"{'Mit Skalierung (2D)':<25} {'2':<10} {ari:.3f}{'':>4} {sil_score:.3f}{'':>8} {get_ari_rating(ari)}")
print(f"{'Ohne Skalierung (2D)':<25} {'2':<10} {ari_unscaled:.3f}{'':>4} {sil_unscaled:.3f}{'':>8} {get_ari_rating(ari_unscaled)}")
print(f"{'Alle Features (13D)':<25} {'13':<10} {ari_all:.3f}{'':>4} {sil_all:.3f}{'':>8} {get_ari_rating(ari_all)}")

print("\n🎯 Erkenntnisse:")
improvement_2d = ((ari - ari_unscaled) / ari_unscaled * 100) if ari_unscaled > 0 else 0
improvement_13d = ((ari_all - ari) / ari * 100) if ari > 0 else 0

print(f"   • Skalierung (2D): {improvement_2d:+.1f}% Verbesserung")
print(f"   • Mehr Features (13D): {improvement_13d:+.1f}% weitere Verbesserung")
print(f"   • Gesamt-Verbesserung: {((ari_all - ari_unscaled) / ari_unscaled * 100):+.1f}%")

print("\n💡 Was bedeutet das?")
if ari_all > ari:
    print("   ✅ Mehr Features = bessere Ergebnisse!")
    print("   ✅ k-Means kann alle 13 chemischen Eigenschaften nutzen")
    print("   ✅ Die zusätzlichen Features enthalten wertvolle Informationen")
else:
    print("   🤔 Mehr Features helfen nicht immer automatisch")
    print("   🤔 Manchmal reichen die wichtigsten Features aus")
    print("   🤔 Curse of dimensionality könnte eine Rolle spielen")

print(f"\n🏆 Bester Ansatz: {'13 Features' if ari_all > max(ari, ari_unscaled) else '2 Features (skaliert)' if ari > ari_unscaled else '2 Features (unskaliert)'}")

## 10. Zusammenfassung und Erkenntnisse

Was haben wir gelernt?

In [ ]:
print("🎓 Wichtige Erkenntnisse aus diesem Notebook:")
print()
print("✅ Was gut funktioniert hat:")
print("   • Wine-Dataset zeigt Skalierungsproblem perfekt")
print("   • k-Means kann die Produzenten teilweise finden")
print("   • Skalierung macht einen messbaren Unterschied")
print()
print("⚠️ Was zu beachten ist:")
print("   • k-Means findet nicht immer exakt die echten Gruppen")
print("   • Verschiedene Features haben verschiedene Skalen")
print("   • Ohne Skalierung dominieren große Zahlen")
print("   • k-Means funktioniert am besten bei runden, gleich großen Clustern")
print()
print("🔧 Praktische Tipps:")
print("   • IMMER zuerst Daten mit StandardScaler() skalieren")
print("   • Verschiedene Features anschauen (nicht nur 2)")
print("   • Verschiedene random_state probieren")
print("   • Ergebnisse immer visualisieren und interpretieren")
print()
print("📈 Unsere Ergebnisse:")
print(f"   • ARI ohne Skalierung (2 Features): {ari_unscaled:.3f}")
print(f"   • ARI mit Skalierung (2 Features): {ari:.3f}")
print(f"   • ARI mit allen 13 Features: {ari_all:.3f}")
print(f"   • Verbesserung durch Skalierung: {((ari - ari_unscaled) / ari_unscaled * 100):+.1f}%" if ari_unscaled > 0 else "   • Deutlicher Unterschied!")
print(f"   • Verbesserung durch mehr Features: {((ari_all - ari) / ari * 100):+.1f}%" if ari > 0 else "   • Deutlicher Unterschied!")
print()
print("🏆 Wichtigste Lernerfahrung:")
print("   1️⃣ Skalierung ist KRITISCH bei verschiedenen Einheiten")
print("   2️⃣ Mehr relevante Features können die Ergebnisse verbessern")
print("   3️⃣ Immer mehrere Ansätze vergleichen und bewerten!")